In [3]:
import torchvision

train_data = torchvision.datasets.CIFAR10(root="./dataset", train=True, transform=torchvision.transforms.ToTensor(), download=True)

test_data = torchvision.datasets.CIFAR10(root="./dataset", train=False, transform=torchvision.transforms.ToTensor(), download=True)

Files already downloaded and verified
Files already downloaded and verified


In [12]:
train_data_size = len(train_data)
train_data_size

50000

In [11]:
test_data_size = len(test_data)
test_data_size

10000

In [6]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [7]:
import torch
from torch import nn
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential
from torch.utils.tensorboard import SummaryWriter

class MyModule(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        # following is same
        self.model = Sequential(
            Conv2d(3, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 64, 5, padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10),
        )
    
    def forward(self, x):
        out = self.model(x)
        return out
    
if __name__ == '__main__':
    myModule = MyModule()
    input = torch.ones((64, 3, 32, 32))
    output = myModule(input)
    print(output.shape)

torch.Size([64, 10])


In [16]:
myModule = MyModule()
myModule.cuda()

writer = SummaryWriter(log_dir="logs")

# loss function
loss_fn = nn.CrossEntropyLoss()
loss_fn.cuda()

# optimizer
learning_rate = 0.01 # 1e-2
optimizer = torch.optim.SGD(myModule.parameters(), lr=learning_rate)

# trainning

# myModule.train() or myModule.eval() if we got dropout or ..., we should add .train() ...

total_train_step = 0
total_test_step = 0
epoch = 10

for i in range(epoch):
    print("epoch {0} beginn: ".format(i+1))
    for data in train_dataloader:
        imgs, targets = data
        imgs = imgs.cuda()
        targets = targets.cuda()
        outputs = myModule(imgs)
        loss = loss_fn(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step += 1
        if total_train_step % 100 == 0:
            print("trainning {0}, Loss: {1}".format(total_train_step, loss.item())) # loss.item() tensor type --> float or double 
            writer.add_scalar("train_loss", loss.item(), total_train_step)

    # testing
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            imgs = imgs.cuda()
            targets = targets.cuda()
            outputs = myModule(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy += accuracy
    print("testing loss: {}".format(total_test_loss))
    print("total accuarcy: {}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuary", total_accuracy/test_data_size, total_test_step)
    total_test_step = total_test_step + 1

    torch.save(myModule, "myModule_{}.pth".format(i))
    print("model saved ...")

writer.close()

epoch 1 beginn: 
trainning 100, Loss: 2.2900452613830566
trainning 200, Loss: 2.283712148666382
trainning 300, Loss: 2.254767417907715
trainning 400, Loss: 2.1484146118164062
trainning 500, Loss: 2.045743942260742
trainning 600, Loss: 1.9902491569519043
trainning 700, Loss: 2.0339903831481934
testing loss: 319.6026087999344
total accuarcy: 0.2733999788761139
model saved ...
epoch 2 beginn: 
trainning 800, Loss: 1.8658075332641602
trainning 900, Loss: 1.8569884300231934
trainning 1000, Loss: 1.9118919372558594
trainning 1100, Loss: 1.958569049835205
trainning 1200, Loss: 1.6741749048233032
trainning 1300, Loss: 1.6373035907745361
trainning 1400, Loss: 1.7563551664352417


KeyboardInterrupt: 